# 발표 소단원 리스트

14-2,14-6,14-10,14-14

# 문항별 자료 

- (14-2) : Patching Objects in Unit Tests

- (14-6) : Handling Multiple Exceptions

- (14-10) : Reraising the Last Exception

- (14-14) : Making Your Programse run faster (without C-extension, JIT complier)

### (14-2) Patching Objects in Unit Tests
- 

In [31]:
from unittest.mock import patch
import os

def rm(file_name):
    os.remove(file_name)

## 실제로 사라지지 않음 -> 동작 하는지만 살펴봄!!
with patch('__main__.rm') as mock_func:
    rm('test.txt')
    mock_func.assert_called_with('test.txt')

In [33]:
## 추가적으로 decorator 사용하는 방식
@patch('__main__.rm')
def test1(file_name,mock_func):
    rm(file_name)
    mock_func.assert_called_with(file_name)
    
test1('test.txt')

In [19]:
from unittest.mock import patch

x = 50

with patch('__main__.x',35):
    print(x)

35


In [20]:
x

50

In [24]:
def calc_result(num):
    print(num**2)

In [25]:
from unittest.mock import patch

x = 50

with patch('__main__.x',35):
    ## 이 환경 속에서 함수의 특정 인자 값을 원하는 것으로 바꿔가면서 테스트 가능!!
    print(x)
    calc_result(x)

35
1225


### 14-6) Handling Multiple Exceptions
- 여러개 에러 처리 방법
    1. 동시에 에러 받기
    2. except 여러개 쓰기 : 위에꺼부터 순차적으로 실행 (위에서 에러 발생하면 아래 실행 안 됨)
    3. 여러개 에러의 상위 클래스로 받기
 

In [59]:
## 1. 동시에 받기
from urllib.error import *
from socket import timeout

try:
    raise ValueError
except (ValueError,URLError,timeout):
    print('total error')
    
    
## 2. except 여러개 쓰기 : 에러마다 처리 다르게 해주고 싶을 때
try:
    raise timeout
except(URLError,ValueError):
    print('two error')
except(timeout):
    print('Only Timeout error')

## 3. 여러개 상위 클래스로 받기
try:
    f = open('no_such_file.txt')
except (FileNotFoundError,PermissionError):
    print('no file')
    
## 대신해서
try:
    f = open('no_such_file.txt')
except OSError:
    print('no file')

total error
Only Timeout error
no file
no file


In [63]:
## 에러에 이름 달아서 세부사항 꺼내기
try:
    f = open('no_such_file.txt')
except OSError as e:
    print(e.errno)

2


In [66]:
try:
    f = open('no_such_file.txt')
except FileNotFoundError:
    print('filenotfounderror')
except OSError:
    print('OSError')

filenotfounderror


In [67]:
try:
    f = open('no_such_file.txt')
except OSError:
    print('OSError')
except FileNotFoundError:
    print('filenotfounderror')

OSError


In [68]:
FileNotFoundError.__mro__## 상위 클래스 BaseException까지 써볼 수 있음

(FileNotFoundError, OSError, Exception, BaseException, object)

### 14-10) Reraising the Last Exception
- except 처리하고 raise 한 번 더 불러주기..

In [69]:
try:
    int('N/A')
except ValueError:
    ## logging
    print('saved in log file')
    raise

saved in log file


ValueError: invalid literal for int() with base 10: 'N/A'

### (14-14) Making Your Programse run faster (without C-extension, JIT complier)
    
- for loop나 중요한 부분에 신경쓰기 (minor한 부분은 무시해도 좋음) 
    - 1. local 변수를 쓰는게 더 빠름 : 함수도 마찬가지
    - 2. 로컬화
    - 3. decorator 피하기
    - 4. built in data type 이용하기 (list,set,dict,tuples,string)

In [88]:
## 1. 로컬 변수 쓰기
import time
start = time.time()
with open('test.txt') as f:
    a= f.readlines()
    print([s[0] for s in a])
time.time() - start

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


0.005220174789428711

In [82]:
def main(filename):
    with open(filename) as f:
        a= f.readlines()
        print([s[0] for s in a])

In [89]:
import time
start = time.time()

main('test.txt')
time.time() - start

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


0.0051920413970947266

In [91]:
## 2. 로컬화
import math
sqrt = math.sqrt
result=[]
result_append = result.append

In [94]:
## 3. 데코레이터 피하기
class A:
    def __init__(self,x,y):
        self.x = x
        self.y = y
        
    @property
    def y(self):
        return self._y
    
    @y.setter
    def y(self,value):
        self._y = value

In [95]:
from timeit import timeit
a = A(1,2)
timeit('a.x','from __main__ import a')

0.03317560000141384

In [96]:
timeit('a.y','from __main__ import a')

0.12396319999970729

In [ ]:
## 4. built in data type 이용하기 (list,set,dict,tuples,string)
